In [647]:
import itertools
from collections import Counter
from functools import lru_cache
import util

In [434]:
KEYWORDS = {'FREE', 'FARE', 'AREA', 'REEF'}
LETTERS = set(Counter(itertools.chain(*KEYWORDS)).keys())
WORDLEN = len(next(iter(KEYWORDS)))
LIMIT = 15
print(LETTERS, KEYWORDS, WORDLEN, LIMIT)

{'R', 'E', 'A', 'F'} {'FREE', 'FARE', 'AREA', 'REEF'} 4 15


In [587]:
def find_overlaps(s1, s2):
    return [i for i in range(len(s1)) if s2.startswith(s1[i:])]

def apply_overlaps(s1, s2, overlaps):
    return [s1[:i] + s2 for i in overlaps]

def limit_len(words, limit):
    return [w for w in words if len(w) <= limit]

import re

def count_overlapping(text, search_for):
    return len(re.findall(search_for, text))

_keyword_res = {re.compile(rf'(?=({k}))') for k in KEYWORDS}
def is_minimal(w):
    return all(count_overlapping(w, k) == 1 for k in _keyword_res)

def is_safe(w):
    return all(count_overlapping(w, k) == 0 for k in _keyword_res)

In [878]:
@lru_cache(maxsize=None)
def safe_suffixes(prev, n):
    if n <= 0: return 1
    words = {(prev+l)[-WORDLEN:] for l in LETTERS}
    return sum(safe_suffixes(w, n-1) if n > 0 else 1 for w in words if w not in KEYWORDS)

@lru_cache(maxsize=None)
def safe_prefixes(post, n):
    if n <= 0: return 1
    words = {(l+post)[:WORDLEN] for l in LETTERS}
    return sum(safe_prefixes(w, n-1) if n > 0 else 1 for w in words if w not in KEYWORDS)

@lru_cache(maxsize=None)
def _safe_infixes(prev, post, n):
    if n <= 0: 
        return 1 if is_safe(prev[1:] + post[:-1]) else 0
    words = {(prev+l)[-WORDLEN:] for l in LETTERS}
    return sum(safe_infixes(w, post, n-1) for w in words if w not in KEYWORDS)    

def safe_infixes(prev, post, n):
    return _safe_infixes(prev[-WORDLEN:], post[:WORDLEN], n)

def f(n):
    return set(''.join(ls) for ls in itertools.product(*[LETTERS]*n) if is_minimal(''.join(ls)))

In [891]:
solutions_10 = f(10)
#solutions_11 = f(11)
solutions_12 = f(12)

In [892]:
len(f(12))

450

In [584]:
safe_suffixes('REA', 2)

16

In [585]:
safe_prefixes('FRE', 4)

249

In [597]:
safe_infixes('REE', 'A', 0)

1

In [522]:
re.findall(r'(?=(AREA))', 'AREAREA')

['AREA', 'AREA']

In [866]:
def overlap_ordered(words, limit):
    overlaps = [words[0]]
    for word in words[1:]:
        new_overlaps = []
        for prefix in overlaps:
            #new_overlaps.append(prefix + word)
            new_overlaps.extend(apply_overlaps(prefix, word, find_overlaps(prefix, word)))
        new_overlaps = limit_len(new_overlaps, limit)
        if len(new_overlaps) == 0:
            return []
        overlaps = new_overlaps
    return overlaps
                    
def overlapping_solutions(keywords, limit):
    overlaps = set()
    for words in itertools.permutations(keywords):
        overlaps.update(overlap_ordered(words, maxlen))
    return {o for o in overlaps if is_minimal(o)}

def partitions(xs):
    if len(xs) == 1:
        yield [ xs ]
        return

    first = xs[0]
    for smaller in partitions(xs[1:]):
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        yield [ [ first ] ] + smaller

def placements(keywords, limit):
    placements = set()
    for partition in partitions(keywords):
        for partition in itertools.product(*(itertools.permutations(c) for c in partition)):
            for placement in itertools.product(*(overlap_ordered(ws, limit) if len(ws) > 1 else ws for ws in partition)):
                if num_blanks(placement, limit) < 0: continue
                placements.update(itertools.permutations(placement))
    return placements

def num_blanks(placement, limit):
    return limit - sum(len(w) for w in placement)

def distribute_blanks(blanks, gaps):
    if gaps == 1: 
        return [[blanks]]
    elif blanks <= 0:
        return [[blanks]*gaps]
    solutions = []
    for i in range(blanks+1):
        solutions.extend([i] + remainder for remainder in distribute_blanks(blanks - i, gaps - 1))
    return solutions

In [726]:
list(placements(list(KEYWORDS), 10))

[('FREEFAREA',), ('FREEF', 'FAREA'), ('FAREA', 'FREEF')]

In [914]:
solutions = 0
LIMIT = 30

for placement in placements(list(KEYWORDS), LIMIT):
    blanks = num_blanks(placement, LIMIT)
    solution = ''.join(placement)
    # Multiple parts to place amid blanks.
    for dist in distribute_blanks(blanks, len(placement)+1):
        terms = []
        for i, blanks in enumerate(dist):
            if i == 0:
                term = safe_prefixes(placement[0][:WORDLEN], blanks)
            elif i == len(placement):
                term = safe_suffixes(placement[-1][-WORDLEN:], blanks)
            else:
                term = safe_infixes(placement[i-1], placement[i], blanks)
            terms.append(term)
        results = util.product(terms)
#             print(solution, placement, dist, '->', results)
        solutions += results

print(solutions)

644997092988678


In [906]:
len(solutions_12)

450

In [6]:
for k1, k2 in itertools.combinations(keywords, 2):
    print(k1, k2)
    for i in find_overlaps(k1, k2):
        print(' ', k1[:i] + k2)
    for j in find_overlaps(k2, k1):
        print(' ', k2[:i] + k1)

FREE FARE
FREE AREA
FREE REEF
  FREEF
  RFREE
FARE AREA
  FAREA
FARE REEF
  FAREEF
  REFARE
AREA REEF


['FREE', 'FARE', 'AREA', 'REEF']
['FREEFAREAREEF', 'FREEFAREEFAREA', 'FREEAREAFAREEF', 'FREEAREAREEFARE', 'FREEREEFFAREA', 'FREEREEFAREAREA', 'FREEREEFAREA', 'FREEFFAREAREA', 'FREEFFAREA', 'FREEFAREAREA', 'FREEFAREA', 'FREEFAREAFARE', 'FAREFREEFAREA', 'FAREAREAFREEF', 'FAREAFREEREEF', 'FAREAFREEF', 'FAREAREAREEFREE', 'FAREAREEFFREE', 'FAREAREEFREE', 'FAREREEFREEAREA', 'FAREEFFREEAREA', 'FAREEFREEAREA', 'FAREEFAREAFREE', 'AREAFREEFAREEF', 'AREAFREEREEFARE', 'AREAFREEFFARE', 'AREAFREEFARE', 'AREAFAREFREEF', 'AREAFAREREEFREE', 'AREAFAREEFFREE', 'AREAFAREEFREE', 'AREAREEFREEFARE', 'AREAREEFAREFREE', 'REEFFREEFAREA', 'REEFREEFAREAREA', 'REEFREEFAREA', 'REEFREEAREAFARE', 'REEFAREFREEAREA', 'REEFFAREAFREE', 'REEFAREAREAFREE', 'REEFAREAFREE']
41


In [57]:
def minimize(overlaps):
    minimal_overlaps = set()
    for w1 in overlaps:
        if all((w1 == w2 or w2 not in w1) for w2 in overlaps):
            minimal_overlaps.add(w1)
    return minimal_overlaps

minimal_overlaps = minimize(overlaps)
print(len(minimal_overlaps))
print(minimal_overlaps)

30
{'FREEFFAREA', 'FREEFAREEFAREA', 'FREEAREAREEFARE', 'FAREREEFREEAREA', 'REEFREEAREAFARE', 'AREAFREEREEFARE', 'FREEREEFAREA', 'AREAFREEFARE', 'AREAFAREREEFREE', 'FREEREEFFAREA', 'FAREAREEFREE', 'AREAFREEFFARE', 'FAREAFREEREEF', 'REEFFAREAFREE', 'FAREAREAFREEF', 'AREAREEFAREFREE', 'AREAFAREFREEF', 'REEFAREAREAFREE', 'FAREAFREEF', 'REEFAREFREEAREA', 'FAREAREAREEFREE', 'AREAREEFREEFARE', 'AREAFAREEFFREE', 'FREEAREAFAREEF', 'AREAFAREEFREE', 'FAREEFFREEAREA', 'REEFAREAFREE', 'FREEFAREA', 'FAREEFREEAREA', 'FAREAREEFFREE'}


In [10]:
from math import perm

count = 0
for overlap in minimal_overlaps:
    r = maxlen - len(overlap)
    c = perm(r+1) * 4 ** r
    print(overlap, r, c)

FREEFFAREA 5 737280
FREEFAREEFAREA 1 8
FREEAREAREEFARE 0 1
FAREREEFREEAREA 0 1
REEFREEAREAFARE 0 1
AREAFREEREEFARE 0 1
FREEREEFAREA 3 1536
AREAFREEFARE 3 1536
AREAFAREREEFREE 0 1
FREEREEFFAREA 2 96
FAREAREEFREE 3 1536
AREAFREEFFARE 2 96
FAREAFREEREEF 2 96
REEFFAREAFREE 2 96
FAREAREAFREEF 2 96
AREAREEFAREFREE 0 1
AREAFAREFREEF 2 96
REEFAREAREAFREE 0 1
FAREAFREEF 5 737280
REEFAREFREEAREA 0 1
FAREAREAREEFREE 0 1
AREAREEFREEFARE 0 1
AREAFAREEFFREE 1 8
FREEAREAFAREEF 1 8
AREAFAREEFREE 2 96
FAREEFFREEAREA 1 8
REEFAREAFREE 3 1536
FREEFAREA 6 20643840
FAREEFREEAREA 2 96
FAREAREEFFREE 2 96


In [ ]:
# Failed 

In [370]:
from functools import lru_cache

minlen = 4
os = sorted(minimize(all_overlaps(keywords, maxlen)), key=len)
letters = 'FARE'
s1, s2 = None, None
@lru_cache(maxsize=None)
def num_safe_strings(n):
    if n <= minlen:
        return 4**n - sum(len(w) == n for w in words)
    base = (num_safe_strings(n - 1) * 4 - 
            num_safe_strings(n - minlen) * len(keywords))
    print('add', num_safe_strings(n - 1) * 4)
    sub = {w1+w2 for w1 in safe_strings(n-minlen) for w2 in keywords}
    print('sub', num_safe_strings(n - minlen) * len(keywords), len(sub))
    extras = set()
    for w in keywords:
        for o1 in os:
            if not o1.endswith(w) or len(o1) > n: continue
            base += num_safe_strings(n - len(o1))
            print('add', w, o1, len(o1), num_safe_strings(n - len(o1)))
            for s in safe_strings(n - len(o1)):
#                 if s+o1 in extras or s+o1 in sub:
#                     print('AHHH!', s+o1, w, o1)
#                 else:
                print(n-len(o1))
                extras.add(s+o1)
    prev = {s+l for s in safe_strings(n-1) for l in letters}
    global s1
    s1 = extras
    print(prev & extras, len(prev), len(extras))

#             for o2 in os:
#                 if o1 == o2 or not o2.endswith(o1) or len(o2) > n: continue
#                 base -= num_safe_strings(n - len(o2))
#                 print('sub', num_safe_strings(n - len(o2)))
#                 break
    return base

@lru_cache(maxsize=None)
def safe_strings(n):
    if n <= minlen:
        everything = set(''.join(ls) for ls in itertools.product(*[letters]*n))
        return everything - set(keywords)
    everything = {s+l for s in safe_strings(n - 1) for l in letters}
    print('add', len(everything))
    sub1 = {w for w in everything if any(w.endswith(kw) for kw in keywords)}
    sub2 = {w1+w2 for w1 in safe_strings(n-minlen) for w2 in keywords}
    print('sub', len(sub1), len(sub2))
    sub3 = set()
    for w in keywords:
        for o1 in os:
            if not o1.endswith(w) or len(o1) > n: continue
            sub3.update(s+o1 for s in safe_strings(n - len(o1)))
            #print('add', w, o1, len(o1), num_safe_strings(n - len(o1)))
    print(sub2 - (sub1 | sub3))
    global s2
    s2 = sub2 - sub1
    return everything - sub1

In [371]:
s1 - s2

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

In [372]:
len(os)

17

In [458]:
len(safe_strings(5))

994

In [355]:
1008-857

151

In [459]:
num_safe_strings(5)

994

In [322]:
61868-1008+167

61027

In [282]:
total = 0
letters = 'FARE'
everything = set()
for ls in itertools.product(*[letters]*8):
    w1 = ''.join(ls)
    if all(w2 not in w1 for w2 in keywords):
        everything.add(w1)
        total += 1
print(total, len(everything))

61011 61011


In [198]:
for w in sorted(everything):
    if any(w.startswith(word) for word in keywords):
        everything.remove(w)
print(len(everything))

15467


In [258]:
[(o, len(o)) for o in os]

[('FREE', 4), ('FARE', 4), ('AREA', 4), ('REEF', 4)]

In [164]:
[(o, len(o)) for o in os]

[('REEFFREE', 8),
 ('REEFFARE', 8),
 ('AREAFREE', 8),
 ('FAREEF', 6),
 ('AREAREEF', 8),
 ('AREAFARE', 8),
 ('REEFREE', 7),
 ('REEFARE', 7),
 ('FREEAREA', 8),
 ('FREEF', 5),
 ('FREEREEF', 8),
 ('FAREREEF', 8),
 ('FAREFREE', 8),
 ('FAREA', 5)]

In [96]:
for w1, w2 in itertools.product(words, words):
    if w1 == w2: continue
    print(overlap_ordered([w1, w2], maxlen))

['REEFAREA']
['REEFFARE', 'REEFARE']
['REEFFREE', 'REEFREE']
['AREAREEF']
['AREAFARE']
['AREAFREE']
['FAREREEF', 'FAREEF']
['FAREAREA', 'FAREA']
['FAREFREE']
['FREEREEF', 'FREEF']
['FREEAREA']
['FREEFARE']
